# Es necesario informar el días desde que se van a recoger datos y el número de dias, así como el código del aeropueto
Buscar aquí: https://www.wunderground.com/history

In [ ]:
#Parámetros

#aeropuerto = 'LIPC' #Cervia
#aeropuerto = 'LEMG' #Marbella
#aeropuerto = 'EDDF' #Frankfurt
#aeropuerto = 'LEPA' #Palma
#aeropuerto = 'GCRR' #Lanzaorte
#aeropuerto = 'LEVT' #Vitoria
#aeropuerto = 'LELC' #Murcia. San Javier
#aeropuerto = 'EDDS' #Nuremberg
aeropuerto = 'LEMD' #MadridBarajas


anio_ini = 2017
mes_ini = 9
dia_ini = 1
num_dias = 30

anios = 5

In [3]:
#Dependecias Externas

import sys
import requests
from bs4 import BeautifulSoup
import xlsxwriter
import pandas as pd
import time
import datetime
from metar import Metar


In [81]:
#Definición de Funciones

def lista_fecha(anio, mes, dia, dias):
    datelist = pd.date_range(datetime.datetime(anio, mes, dia), periods=dias).tolist()
    result = map(lambda x: x.to_pydatetime().date().strftime('%Y/%m/%d'), datelist)
    return list(result)

def leer_meteo(url):
    headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5)",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "accept-charset": "ISO-8859-1,utf-8;q=0.7,*;q=0.3",
    "accept-encoding": "gzip,deflate,sdch",
    "accept-language": "en-US,en;q=0.8",
    }

    #Metemos las cookies para que el html que devuelva sea el que necesitamos
    cook = dict(Prefs='FAVS:1|WXSN:1|PWSOBS:1|WPHO:1|PHOT:1|RADC:0|RADALL:0|HIST0:NULL|GIFT:1|SHOWMETAR:1|')
    
    r = requests.get(url, headers=headers, cookies=cook)
    
    if r.status_code != 200:
        print("request denied")
        print(r.status_code)
        sigue = 0

    soup = BeautifulSoup(r.text, 'html.parser')
    lista_info = []
    try:
        table =  soup.select("#obsTable")[0]
        for tr in table.find_all('tr'):
            for tds in tr.find_all('td', attrs={"colspan":"12"}):
                lista_info.append(tds.text)
                #print(tds.text)
        return lista_info
    except:
        pass

def filtrar_metars (lista):
    result = []
    try:
        lista.sort()
    
        for linea in lista:
            if linea[:5] == 'METAR':
                result.append(linea)
        return result
    except:
        pass

def metar_parser(solo_metars, ubicacion, fecha):
    metar = []
    try:
        for element in solo_metars:
            lluvia = 0
            dir_viento = 0
            fuerza_viento = 0
            temp = 0
            
            # Initialize a Metar object with the data from solo_metars
            try:
                obs = Metar(element)
            except:
                pass
            
            estacion = ubicacion

            try:
                h = "00"+str(obs.time.hour)
                m = "00"+str(obs.time.minute)
                hora = fecha+" "+h+":"+m[-2:]
            except:
                hora = ''

            try:
                temp = obs.temp.value('C')                  
            except:
                temp = ''

            try:
                if obs.wind_dir.value()==360.0:
                    dir_viento = 0
                else:
                    dir_viento = obs.wind_dir.value()
            except:
                dir_viento = ''

            try:
                fuerza_viento = obs.wind_speed.value("KMH")
            except:
                fuerza_viento = ''

            try:
                lluvia = obs.precip_1hr.string("in")/0.061024
            except:
                lluvia = ''
            
            #print(estacion, hora, temp, fuerza_viento, dir_viento, lluvia)
            metar.append([estacion, hora, temp, fuerza_viento, dir_viento, lluvia])
        return metar
    except:
        pass


In [82]:
#Ejecucuón

lista_df =[]
for year in range(anios):
    for dia in lista_fecha(anio_ini-year,mes_ini,dia_ini,num_dias):
        print(dia)
        url = 'https://www.wunderground.com/history/airport/'+aeropuerto+'/'+dia+'/DailyHistory.html'
        print(url)
        meteo_list = leer_meteo(url)
        solo_metars = filtrar_metars(meteo_list)
        info = metar_parser(solo_metars, aeropuerto, dia)
        dfinfo = pd.DataFrame(info)
        try:
            dfinfo.columns = ['Estacion','Fecha_Hora','Temp','Fuerza_Viento','Direccion_Viento','Lluvia']
            dfinfo['Fecha_Hora'] = pd.to_datetime(dfinfo['Fecha_Hora'])
        except:
            pass
        
        lista_df.append(dfinfo)
print('fin bucle - guardando')
dfresultado_final = pd.concat(lista_df)
print('guardado - printing')
dfresultado_final.to_excel("metars"+aeropuerto+".xls")
print('fin')


NameError: name 'anios' is not defined

# Probamos a hacer una funcion que recibiendo parámetros de Aeropuerto y fecha devuelva el DataFrame de ese día

In [96]:
def get_tiempo(aeropuerto,dia):
    #Formato de fecha admitida "2018/5/7"
    print(dia)
    print(aeropuerto)
    url = 'https://www.wunderground.com/history/airport/'+aeropuerto+'/'+dia+'/DailyHistory.html'
    print(url)
    meteo_list = leer_meteo(url)
    solo_metars = filtrar_metars(meteo_list)
    info = metar_parser(solo_metars, aeropuerto, dia)
    dfinfo = pd.DataFrame(info)
    try:
        dfinfo.columns = ['Estacion','Fecha_Hora','Temp','Fuerza_Viento','Direccion_Viento','Lluvia']
        dfinfo['Fecha_Hora'] = pd.to_datetime(dfinfo['Fecha_Hora'])
    except:
        pass

    out = dfinfo[dfinfo["Direccion_Viento"]!=""]
    
    return out

In [97]:
test = get_tiempo('LEMD',"2018/5/7")

2018/5/7
LEMD
https://www.wunderground.com/history/airport/LEMD/2018/5/7/DailyHistory.html


In [98]:
test

,Estacion,Fecha_Hora,Temp,Fuerza_Viento,Direccion_Viento,Lluvia
0,LEMD,2018-05-07 22:00:00,16.0,7.407994,180,
1,LEMD,2018-05-07 22:30:00,17.0,7.407994,180,
2,LEMD,2018-05-07 23:00:00,15.0,5.555995,20,
3,LEMD,2018-05-07 23:30:00,14.0,9.259992,0,
4,LEMD,2018-05-07 00:00:00,14.0,9.259992,330,
6,LEMD,2018-05-07 01:00:00,13.0,3.703997,130,
7,LEMD,2018-05-07 01:30:00,13.0,5.555995,60,
10,LEMD,2018-05-07 03:30:00,12.0,7.407994,350,
11,LEMD,2018-05-07 04:30:00,12.0,7.407994,330,
12,LEMD,2018-05-07 05:00:00,11.0,3.703997,330,


In [73]:
obs.time.hour

1